In [153]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import time

In [5]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True, as_supervised=True)

Shuffling and writing examples to C:\Users\ikuchin\tensorflow_datasets\ted_hrlr_translate\pt_to_en\1.0.0.incompleteECTBBA\ted_hrlr_translate-train.tfrecord


Shuffling and writing examples to C:\Users\ikuchin\tensorflow_datasets\ted_hrlr_translate\pt_to_en\1.0.0.incompleteECTBBA\ted_hrlr_translate-validation.tfrecord


Shuffling and writing examples to C:\Users\ikuchin\tensorflow_datasets\ted_hrlr_translate\pt_to_en\1.0.0.incompleteECTBBA\ted_hrlr_translate-test.tfrecord


Dataset ted_hrlr_translate downloaded and prepared to C:\Users\ikuchin\tensorflow_datasets\ted_hrlr_translate\pt_to_en\1.0.0. Subsequent calls will reuse this data.


In [6]:
examples

{'test': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.string)>,
 'train': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.string)>,
 'validation': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.string)>}

In [7]:
train_examples, val_examples = examples['train'], examples['validation']

In [21]:
def print_first(ds, number_of_items = 2):
    for idx, item in zip(range(number_of_items), ds):
        print(item)
print_first(train_examples)

(<tf.Tensor: shape=(), dtype=string, numpy=b'e quando melhoramos a procura , tiramos a \xc3\xbanica vantagem da impress\xc3\xa3o , que \xc3\xa9 a serendipidade .'>, <tf.Tensor: shape=(), dtype=string, numpy=b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .'>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'mas e se estes fatores fossem ativos ?'>, <tf.Tensor: shape=(), dtype=string, numpy=b'but what if it were active ?'>)


In [30]:
tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus((en.numpy() for pt, en in train_examples), target_vocab_size = 2**13)
tokenizer_pt = tfds.features.text.SubwordTextEncoder.build_from_corpus((pt.numpy() for pt, en in train_examples), target_vocab_size = 2**13)

In [33]:
tokenizer_en.encode("Transformer is awesome.")

[7915, 1248, 7946, 7194, 13, 2799, 7877]

In [37]:
tokenizer_en.vocab_size

8087

In [176]:
def add_start_stop(lang1, lang2):
    start_id_1 = [tokenizer_en.vocab_size]
    start_id_2 = [tokenizer_pt.vocab_size]
    stop_id_1  = [tokenizer_en.vocab_size + 1]
    stop_id_2  = [tokenizer_pt.vocab_size + 1]
   
#     print(lang1)
    lang1 = start_id_1 + tokenizer_pt.encode(lang1.numpy()) +  stop_id_1
    lang2 = start_id_2 + tokenizer_en.encode(lang2.numpy()) + stop_id_2

    return lang1, lang2

def add_start_stop_wrapper(lang1, lang2):
    lang1, lang2 = tf.py_function(add_start_stop, [lang1, lang2], [tf.int32, tf.int32])
    print("1)", lang2)
    lang1.set_shape([None])
    lang2.set_shape([None])
    print("2)", lang2)
    
    
    return lang1, lang2

MAX_SIZE = 40
def filter_long_seq(lang1, lang2):
    return tf.logical_and(tf.size(lang1) < MAX_SIZE, tf.size(lang2) < MAX_SIZE)
# add_start_stop_wrapper('123', '234')

In [144]:
train_ds = train_examples.map(add_start_stop_wrapper)
print_first(train_ds)

1) Tensor("EagerPyFunc:1", dtype=int32, device=/job:localhost/replica:0/task:0)
2) Tensor("EagerPyFunc:1", shape=(None,), dtype=int32, device=/job:localhost/replica:0/task:0)
(<tf.Tensor: shape=(27,), dtype=int32, numpy=
array([8087,    6,   40, 4092,   57,    3, 1687,    1, 6155,   12,    3,
        461, 6770,   19, 5227, 1088,   97,    1,    5,    8,    3, 4213,
       3408, 7256, 1670,    2, 8088])>, <tf.Tensor: shape=(27,), dtype=int32, numpy=
array([8214,    4,   59,   15, 1792, 6561, 3060, 7952,    1,   15,  103,
        134,  378,    3,   47, 6122,    6, 5311,    1,   91,   13, 1849,
        559, 1609,  894,    2, 8215])>)
(<tf.Tensor: shape=(11,), dtype=int32, numpy=array([8087,   25,    6,   16,  138, 7800, 2004, 2445, 8073,   29, 8088])>, <tf.Tensor: shape=(9,), dtype=int32, numpy=array([8214,   23,   29,   44,   17,   72, 4332,   33, 8215])>)


In [148]:
train_ds = train_examples.map(add_start_stop_wrapper).batch(1)
print_first(train_ds)

1) Tensor("EagerPyFunc:1", dtype=int32, device=/job:localhost/replica:0/task:0)
2) Tensor("EagerPyFunc:1", shape=(None,), dtype=int32, device=/job:localhost/replica:0/task:0)
(<tf.Tensor: shape=(1, 27), dtype=int32, numpy=
array([[8087,    6,   40, 4092,   57,    3, 1687,    1, 6155,   12,    3,
         461, 6770,   19, 5227, 1088,   97,    1,    5,    8,    3, 4213,
        3408, 7256, 1670,    2, 8088]])>, <tf.Tensor: shape=(1, 27), dtype=int32, numpy=
array([[8214,    4,   59,   15, 1792, 6561, 3060, 7952,    1,   15,  103,
         134,  378,    3,   47, 6122,    6, 5311,    1,   91,   13, 1849,
         559, 1609,  894,    2, 8215]])>)
(<tf.Tensor: shape=(1, 11), dtype=int32, numpy=array([[8087,   25,    6,   16,  138, 7800, 2004, 2445, 8073,   29, 8088]])>, <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[8214,   23,   29,   44,   17,   72, 4332,   33, 8215]])>)


In [177]:
train_ds = train_examples.map(add_start_stop_wrapper)
train_ds = train_ds.filter(filter_long_seq)
train_ds = train_ds.padded_batch(2).prefetch(tf.data.experimental.AUTOTUNE)
print_first(train_ds)

1) Tensor("EagerPyFunc:1", dtype=int32, device=/job:localhost/replica:0/task:0)
2) Tensor("EagerPyFunc:1", shape=(None,), dtype=int32, device=/job:localhost/replica:0/task:0)
(<tf.Tensor: shape=(2, 27), dtype=int32, numpy=
array([[8087,    6,   40, 4092,   57,    3, 1687,    1, 6155,   12,    3,
         461, 6770,   19, 5227, 1088,   97,    1,    5,    8,    3, 4213,
        3408, 7256, 1670,    2, 8088],
       [8087,   25,    6,   16,  138, 7800, 2004, 2445, 8073,   29, 8088,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0]])>, <tf.Tensor: shape=(2, 27), dtype=int32, numpy=
array([[8214,    4,   59,   15, 1792, 6561, 3060, 7952,    1,   15,  103,
         134,  378,    3,   47, 6122,    6, 5311,    1,   91,   13, 1849,
         559, 1609,  894,    2, 8215],
       [8214,   23,   29,   44,   17,   72, 4332,   33, 8215,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0

In [179]:
max_idx = 0
start_time = time.time()
for idx, item in enumerate(train_ds):
    max_idx = idx
print(max_idx, "time taken:", time.time() - start_time)

22248 time taken: 28.674888134002686


In [167]:
max_idx = 0
start_time = time.time()
for idx, item in enumerate(train_ds):
    max_idx = idx
print("number of items:", max_idx, "time taken:", time.time() - start_time)

number of items: 25892 time taken: 26.984004974365234


In [195]:
print_first(train_ds)

(<tf.Tensor: shape=(2, 27), dtype=int32, numpy=
array([[8087,    6,   40, 4092,   57,    3, 1687,    1, 6155,   12,    3,
         461, 6770,   19, 5227, 1088,   97,    1,    5,    8,    3, 4213,
        3408, 7256, 1670,    2, 8088],
       [8087,   25,    6,   16,  138, 7800, 2004, 2445, 8073,   29, 8088,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0]])>, <tf.Tensor: shape=(2, 27), dtype=int32, numpy=
array([[8214,    4,   59,   15, 1792, 6561, 3060, 7952,    1,   15,  103,
         134,  378,    3,   47, 6122,    6, 5311,    1,   91,   13, 1849,
         559, 1609,  894,    2, 8215],
       [8214,   23,   29,   44,   17,   72, 4332,   33, 8215,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0]])>)
(<tf.Tensor: shape=(2, 26), dtype=int32, numpy=
array([[8087,   25,   66,   13,  342,    3, 5278, 7990,    4,   38, 3625,
        8072,    2, 8088,    